In [37]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [38]:
!pip install -q xgboost optuna
print('done')

done


In [53]:
%%time
import numpy as np, pandas as pd, optuna, xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 37.4 µs


In [40]:
data_train = pd.read_csv('/kaggle/input/playground-series-s5e3/train.csv')
data_test = pd.read_csv('/kaggle/input/playground-series-s5e3/test.csv')
data_train.head(2)

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1


In [41]:
data_test.head(2)

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
1,2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3


In [42]:
print(f"{data_train.shape}\n{data_test.shape}")

(2190, 13)
(730, 12)


In [43]:
for col in data_train.columns.to_list():
    print(f"{col} : {data_train[col].isna().sum()}")

id : 0
day : 0
pressure : 0
maxtemp : 0
temparature : 0
mintemp : 0
dewpoint : 0
humidity : 0
cloud : 0
sunshine : 0
winddirection : 0
windspeed : 0
rainfall : 0


In [44]:
for col in data_test.columns.to_list():
    print(f"{col} : {data_test[col].isna().sum()}")

id : 0
day : 0
pressure : 0
maxtemp : 0
temparature : 0
mintemp : 0
dewpoint : 0
humidity : 0
cloud : 0
sunshine : 0
winddirection : 1
windspeed : 0


In [45]:
data_test['winddirection'] = data_test['winddirection'].fillna(data_test['winddirection'].mean())
print('done')

done


In [46]:
data_train = data_train.drop(['id'], axis=1)
print('done')

done


In [47]:
for col in data_train.columns.to_list():
    if col!='rainfall':
        data_train[[col]] = MinMaxScaler().fit_transform(data_train[[col]])
data_train.head(2)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0.000000,0.516854,0.421875,0.547718,0.616279,0.729630,0.813559,0.877551,0.090909,0.172414,0.232305,1
1,0.002747,0.575843,0.226562,0.394191,0.457364,0.581481,0.949153,0.908163,0.000000,0.137931,0.317604,1


In [48]:
for col in data_test.columns.to_list():
    if col!='id':
        data_test[[col]] = MinMaxScaler().fit_transform(data_test[[col]])
data_test.head(2)

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,0.000000,0.605590,0.355634,0.382239,0.341365,0.558052,0.966102,0.99,0.0,0.137931,0.36
1,2191,0.002747,0.512422,0.355634,0.409266,0.465863,0.565543,0.983051,0.99,0.0,0.137931,0.56


In [49]:
X_test = data_test.iloc[:,1:]
X_test.head(2)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,0.000000,0.605590,0.355634,0.382239,0.341365,0.558052,0.966102,0.99,0.0,0.137931,0.36
1,0.002747,0.512422,0.355634,0.409266,0.465863,0.565543,0.983051,0.99,0.0,0.137931,0.56


In [50]:
X_train, X_valid, y_train, y_valid = train_test_split(
    data_train.iloc[:,:-1],
    data_train.iloc[:,-1],
    test_size=0.2
)
print('done')

done


In [51]:
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 500), 
        'max_depth': trial.suggest_int('max_depth', 3, 20),  
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 200),
    }
    forest = RandomForestClassifier(**params)
    forest.fit(X_train, y_train)
    predictions = forest.predict(X_valid)
    return accuracy_score(predictions, y_valid)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

forest_best = study.best_params
print('done')

done


In [54]:
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
def objective(trial):
  params = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
      'max_depth': trial.suggest_int('max_depth', 3, 10),
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
      'subsample': trial.suggest_float('subsample', 0.5, 1.0),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
      'gamma': trial.suggest_float('gamma', 0, 5),
      'alpha': trial.suggest_float('alpha', 0, 10),
      'lambda': trial.suggest_float('lambda', 0, 10),
  }
  boosting = xgb.XGBClassifier(
      **params,
      random_state=42,
      eval_metric='logloss'
  )
  boosting.fit(X_train, y_train)
  predictions = boosting.predict(X_valid)
  return accuracy_score(predictions, y_valid)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

boosting_best = study.best_params
print('done')

done


In [55]:
forest = RandomForestClassifier(**forest_best)
boost = xgb.XGBClassifier(**boosting_best)
forest.fit(X_train, y_train)
boost.fit(X_train, y_train)
print('done')

done


In [56]:
print(
    f"1. {accuracy_score(forest.predict(X_valid), y_valid)}\n"
    f"2. {accuracy_score(boost.predict(X_valid), y_valid)}"
)

1. 0.8584474885844748
2. 0.8561643835616438


In [57]:
preds_1 = forest.predict(X_test)
preds_2 = boost.predict(X_test)
print('done')

done


In [58]:
output = pd.DataFrame({'id': data_test['id'],
                       'rainfall': preds_2})
output.to_csv('submission.csv', index=False)
print('done')

done
